<a id=0></a>
# 9.Regression Models
※　代表的な回帰分析モデルの紹介  
※　モデルの評価、交差検証、パラメータのグリッドサーチ

---
### [1. 探索的データ分析（EDA）](#1)
### [2. LinearRegression](#2)
### [3. RandomForestRegressor](#3)
### [4. KNeighborsRegressor](#4)
### [5. モデルの評価](#5)
### [6. クロスバリデーションとグリッドサーチ](#6)
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(context='talk', style='darkgrid', font='MS GOTHIC')

---
<a id=1></a>
[Topへ](#0)

---
## 1. 探索的データ分析（EDA）¶

* 糖尿病のデータセット
* 分布の確認
* 相関関係
* トレインデータ、テストデータ

糖尿病のデータセット

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
diabetes = load_diabetes()

In [ ]:
# このようにndarrayとして取り出して使っていくこともできる
X, y = load_diabetes(return_X_y=True)

In [ ]:
X[:5]

In [ ]:
Xy = np.concatenate([X, y.reshape(-1, 1)], axis=1)

In [ ]:
Xy.shape

In [ ]:
type(diabetes['feature_names'])
# listであるため、足し合わせで合成することができる

In [ ]:
df = pd.DataFrame(Xy, columns=diabetes['feature_names'] + ['target'])
df.head()

In [ ]:
print(diabetes.DESCR)

In [ ]:
df.info()

In [ ]:
df.describe()
# meanはほぼ０で揃っている

In [ ]:
# カテゴリカルなsexを確認
df['sex'].unique()

分布の確認

In [ ]:
def plot_data(feature):
    fig, axes = plt.subplots(1, 3, figsize=(16, 4), tight_layout=True)
    # sex別に分布を確認
    sns.histplot(data=df, x=feature, hue='sex', multiple='dodge', ax=axes[0])
    # 長いレジェンドを非表示に
    axes[0].legend([], frameon=False)
    # sex別に分布、外れ値を確認
    sns.boxplot(data=df, x='sex', y=feature, width=0.5, ax=axes[1])
    # 長いラベルを非表示に
    axes[1].set_xticklabels([])
    # targetとの関係を確認
    sns.scatterplot(data=df, x=feature, y=y, hue='sex', alpha=0.7, ax=axes[2])
    axes[2].legend([], frameon=False)
    plt.suptitle(feature)
    plt.show()

In [ ]:
for feature in df.columns:
    plot_data(feature)

相関関係

In [ ]:
plt.figure(figsize=(10, 9))
sns.heatmap(df.corr(), annot=True, annot_kws={'size':14}, cmap='Greens', square=True, linewidth=1, vmin=-1)
plt.show()

性別で分けてみる

In [ ]:
df_s0 = df[df['sex'] > 0].drop(columns=['sex'])
df_s1 = df[df['sex'] < 0].drop(columns=['sex'])
# 動画の中でaxis=0を指定しているのは間違いです
# axisを指定する場合は以下のような表現となります
# df_s0 = df[df['sex'] > 0].drop('sex', axis=1)

In [ ]:
df_s0.head()

トレインデータ、テストデータ

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17, shuffle=True)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

In [ ]:
X0_train, X0_test, y0_train, y0_test = train_test_split(df_s0.iloc[:, :-1], df_s0.iloc[:, -1], test_size=0.3, random_state=17)

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(df_s1.iloc[:, :-1], df_s1.iloc[:, -1], test_size=0.3, random_state=17)

---
<a id=2></a>
[Topへ](#0)

---
## 2. LinearRegression

* モデルのインポート  
    https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html  
* モデルインスタンスの作成
* モデルの構築・属性・予測・スコア
* 性別ごとのデータで試してみる
---

モデルのインポート

In [ ]:
from sklearn.linear_model import LinearRegression

モデルインスタンスの作成

In [ ]:
# パラメータの変更はほぼ必要ない
lr_model = LinearRegression()

モデルの構築・属性・予測・スコア

In [ ]:
# 訓練データに対して適合、学習させる
lr_model.fit(X_train, y_train)

In [ ]:
# 予測を行う
lr_pred = lr_model.predict(X_test)
lr_pred

In [ ]:
# 係数、傾き
lr_model.coef_

In [ ]:
# 切片
lr_model.intercept_

In [ ]:
test = np.array([ 0.7076875, -0.04464164, 0.01211685, 0.04252958, 0.07135654, 0.0534871 , 0.05232174, -0.00259226,  0.02539313, -0.0052198 ]).reshape(1, -1)

pred_0 = lr_model.predict(test)
pred_0

In [ ]:
# y = a1x1 + a2x2 + ... + a10x10 + b
pred_1 = lr_model.intercept_  # 切片だけ

# 係数＊値を足し合わせていく
for i in range(10):
    pred_1 += lr_model.coef_[i] * test[0, i]

pred_1

実測値と予測値の比較

In [ ]:
df_result = pd.DataFrame()
df_result['true_value'] = y_test
df_result['prediction'] = lr_pred
df_result['residual'] = y_test - lr_pred
# true_valueで昇順に並べ替え
df_result_sorted = df_result.sort_values('true_value', ignore_index=True)

In [ ]:
plt.figure(figsize=(16, 6))
sns.scatterplot(data=df_result_sorted, x=df_result_sorted.index, y='true_value', color='#c00')
sns.scatterplot(data=df_result_sorted, x=df_result_sorted.index, y='prediction', color='#0c0')
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
sns.scatterplot(data=df_result_sorted, x=df_result_sorted.index, y='residual', color='#0c0')
plt.axhline(0, color="#f00")
plt.show()

In [ ]:
# このスコアはR二乗値、R２スコア
lr_model.score(X_test, y_test)

In [ ]:
# fitに用いたデータに対しても過度に高い数値にはならなかった
lr_model.score(X_train, y_train)

性別ごとのデータで試してみる

In [ ]:
# こちらの性別ではテストでも0.547という結果を得ることができた
lr_model_s0 = LinearRegression()
lr_model_s0.fit(X0_train, y0_train)
lr_model_s0.score(X0_test, y0_test), lr_model_s0.score(X0_train, y0_train)

In [ ]:
lr_model_s1 = LinearRegression()
lr_model_s1.fit(X1_train, y1_train)
lr_model_s1.score(X1_test, y1_test), lr_model_s1.score(X1_train, y1_train)

---
<a id=3></a>
[Topへ](#0)

---
## 3. RandomForestRegressor

* モデルのインポート  
    https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html 
* モデルインスタンスの作成
* モデルの構築・属性・予測・スコア
* パラメータを変更してみる
* Treeの可視化
---

モデルのインポート

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# アンサンブルとは複数人で演奏すること

モデルインスタンスの作成

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=100,   # DecisionTreeRegressorの数
    max_depth=None,   # 分岐を最大何回まで行うか
    min_samples_split=2,    # サンプルが最小いくつまで分岐を行うか（最小であり、達する前に止まることもある）
    min_samples_leaf=1,   # 末端での最小のサンプル数
    max_features=1.0,   # 最大いくつの特徴量を使うか。「最大」なので1.0の場合はすべて使うことも1割を使うこともある
    bootstrap=True,   # 多様性を増やすためのサンプルのランダムな割り当て
    random_state=17,
    max_samples=None   # bootstrap=Trueの場合のサンプルの最大数。0-1.0で指定すればよい
)

モデルの構築・属性・予測・スコア

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
rf_pred = rf_model.predict(X_test)
rf_pred[:5]

In [ ]:
# estimatorを確認
rf_model.base_estimator_

In [ ]:
# 重要度の高さを取得
# index=8, 's5'が最も重要のよう
rf_model.feature_importances_

In [ ]:
rf_model.score(X_test, y_test)

In [ ]:
rf_model.score(X_train, y_train)
# 訓練データに対しては過度に高い精度。新規、未知のデータには精度が低い => 過学習(overfitting)している
# 汎化性能(新規のデータに対しても同程度の精度がある)が低い

パラメータを変更してみる

In [ ]:
# 特に根拠があるわけではない変更です
rf_model_new = RandomForestRegressor(
    n_estimators=120,   # DecisionTreeRegressorの数
    max_depth=4,   # 分岐を最大何回まで行うか
    min_samples_split=5,    # サンプルが最小いくつまで分岐を行うか（最小であり、達する前に止まることもある）
    min_samples_leaf=1,   # 末端での最小のサンプル数
    max_features=0.6,   # 最大いくつの特徴量を使うか。「最大」なので1.0の場合はすべて使うことも1割を使うこともある
    bootstrap=True,   # 多様性を増やすためのサンプルのランダムな割り当て
    random_state=17,
    max_samples=0.7   # bootstrap=Trueの場合のサンプルの最大数。0-1.0で指定すればよい
)

In [ ]:
rf_model_new.fit(X_train, y_train)
rf_pred_new = rf_model_new.predict(X_test)
rf_pred_new[:5]

In [ ]:
rf_model_new.score(X_test, y_test)

In [ ]:
rf_model_new.score(X_train, y_train)
# 過学習が抑えられ（？）、さらに汎化性能もわずかに向上したと考えられる

Treeの可視化

In [ ]:
# ひとつ目のDecisionTreeRegressor
rf_model.estimators_[0]

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(18, 20))
plot_tree(rf_model.estimators_[0], max_depth=2, fontsize=24, feature_names=diabetes['feature_names'], filled=True)
# filled:色分け
plt.show()

---
<a id=4></a>
[Topへ](#0)

---
## 4. KNeighborsRegressor

* モデルのインポート  
     https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html  
* モデルインスタンスの作成・スコア
* パラメータを変更してみる
---

モデルのインポート

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

モデルインスタンスの作成・スコア

In [ ]:
kn_model = KNeighborsRegressor(n_neighbors=5, weights='uniform')
# 近傍、類似の5つのデータから数値を予測する
# 'distance'は距離の近い、遠いで重みづけを変える

In [ ]:
kn_model.fit(X_train, y_train)
kn_pred = kn_model.predict(X_test)
kn_pred[:5]

In [ ]:
kn_model.score(X_test, y_test)

In [ ]:
kn_model.score(X_train, y_train)

パラメータを変更してみる

In [ ]:
kn_model_new = KNeighborsRegressor(n_neighbors=12, weights='uniform')

In [ ]:
kn_model_new.fit(X_train, y_train)

In [ ]:
kn_model_new.score(X_test, y_test)

In [ ]:
kn_model_new.score(X_train, y_train)

---
<a id=5></a>
[Topへ](#0)

---
## 5. モデルの評価方法

* 評価指標
* モデルを評価する
* R2スコア(決定係数)とは
---

評価指標  
絶対平均誤差　/　二乗平均誤差　/　絶対パーセント誤差　/　R二乗値（決定係数）

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

モデルを評価する  

In [ ]:
def eval_models(models, X_train, y_train, X_test, y_test):
    for model in models:
        model.fit(X_train, y_train)
        pred = np.round(model.predict(X_test))
        score = np.round(model.score(X_test, y_test), 4)
        mae = np.round(mean_absolute_error(y_test, pred), 4)
        mse = np.round(mean_squared_error(y_test, pred), 4)   # 2乗するため、誤差の大きさが目立つようになる
        mape = np.round(mean_absolute_percentage_error(y_test, pred), 4)   # 予測値のスケールに寄らず、相対的に理解できる
        r2 = np.round(r2_score(y_test, pred), 4)
        print(f'{model}')
        print(f'score : {score}, mae : {mae}, mse : {mse}, mape : {mape}, r2 : {r2}')
        print('================================')
        
# model.score()とr2_score()は同じもの

In [ ]:
models = [lr_model, rf_model_new, kn_model_new]
eval_models(models, X_train, y_train, X_test, y_test)

R2スコア(決定係数)とは

* R2 = 1 - Σ{( true_value(i) - pred(i) ) ** 2} / Σ{( true_value(i) - true_value_mean ) ** 2}
* 1に近いほど回帰式の精度が高い
* もし、すべてを「平均」で予想した場合(最も安易な？予想)　=> R2 = 0
* 0.5以上：予測精度がそれなりに高いと言える
* 0.7以上：予測精度が高い
* 0.9以上：精度が非常に高い。ただし、過学習でないかの検討も必要  
※　絶対的な基準は存在しません。あくまでも目安として用いる指標です

---
<a id=6></a>
[Topへ](#0)

---
## 6. クロスバリデーションとグリッドサーチ

* KFold  
    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
* cross_val_score  
    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
* GridSearchCV  
    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
---

KFold

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
k = KFold(5, shuffle=True, random_state=17)

In [ ]:
set1, set2, set3, set4, set5 = k.split(X)

In [ ]:
# トレイン、テスト
set1

In [ ]:
len(set1[0]), len(set1[1])

In [ ]:
# ８：２で分割される
89 / 442

cross_val_score

In [ ]:
# 5つの組の平均値で比較する
for model in models:
    scores = cross_val_score(model, X, y, cv=k)
    print(f'{model} : {scores.mean()}')
    print('\n')

GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# rf_modelのパラメータを変更する
# max_depth=4, min_samples_split=5, max_features=0.6, max_samples=0.7
params = {
    'max_depth':[2, 6, 10],
    'max_features':[0.5, 0.9],
    'max_samples':[0.5, 0.9],
    'min_samples_split':[3, 6]
}
# 計２４の組み合わせで結果を取得できる

In [ ]:
grid = GridSearchCV(rf_model, param_grid=params)
# return_train_score=Falseでトレインデータのスコアも確認できる

In [ ]:
grid.fit(X, y)

In [ ]:
# データフレームで結果を表示
pd.DataFrame(grid.cv_results_)

In [ ]:
pd.DataFrame(grid.cv_results_).loc[:, 'params':].sort_values('rank_test_score', ascending=True)

In [ ]:
# 最適なパラメータ
grid.best_params_

---
 <a id=4></a>
[Topへ](#0)

---
## 以上
    
---